In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/shelter-animal-outcomes/train.csv.gz")
test = pd.read_csv("/kaggle/input/shelter-animal-outcomes/test.csv.gz")

In [ ]:
display(train,test)

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data["IsName"] = all_data["Name"].isnull()

In [ ]:
def age_to_day(x):
    if pd.isnull(x):
        return -1
    num = int(x.split()[0])
    if "year" in x:
         return num*365
    elif "month" in x:
         return num*30
    elif "week" in x:
        return num*7 
    else:
        return num
     
    

all_data["AgeuponOutcome"]=all_data["AgeuponOutcome"].apply(age_to_day)

In [ ]:
all_data2 = all_data.drop(["ID","OutcomeType","OutcomeSubtype","AnimalID","DateTime","Name"],1)

In [ ]:
all_data2["AgeuponOutcome"].unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,12))
sns.countplot(all_data["IsName"],hue=all_data["OutcomeType"])

In [ ]:
etc = all_data2["Color"].value_counts()[all_data2["Color"].value_counts() < 500].index

In [ ]:
all_data2["Color"] = all_data2["Color"].apply(lambda x:"etc" if x in etc else x)

In [ ]:
all_data2["Color"].value_counts() #원핫 인코딩 해볼까십다

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data2.columns[all_data2.dtypes == object]:
    all_data2[i] = le.fit_transform(list(all_data2[i]))
    

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid, y_train, y_valid = train_test_split(train2,train["OutcomeType"],test_size=0.2, random_state=42,stratify=train["OutcomeType"])

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()
cat.fit(X_train, y_train, eval_set=(X_valid,y_valid),early_stopping_rounds=50)
result = cat.predict_proba(test2)

In [ ]:
result

In [ ]:
pd.Series(cat.feature_importances_, index=train2.columns)

In [ ]:
sub = pd.read_csv("/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz")
sub.iloc[:,1:] = result
sub.to_csv("shelter.csv",index=0)